# Análisis del riesgo de incumplimiento de los prestatarios
Este proyecto consiste en crear un informe para la división de préstamos de un banco.


Los datos son obtenidos directamente del banco e incluyen datos característicos de los clientes así como su historial de incumplimiento. Los datos se analizarán para obtener características a tomar en cuenta al momento de crear una **puntuación de crédito** para un cliente potencial y así poder evaluar la capacidad de este para pagar su préstamo.

Se probaran distintas hipótesis para responder las siguientes preguntas:
- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?

## Inicialización y vista general



In [926]:
# Cargar todas las librerías
import pandas as pd

# Cargar los datos
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Exploración de datos

**El dataset contiene los siguientes datos:**

- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo



In [927]:
#información general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [928]:
#vista previa
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Al analizar la información del conjunto de datos se sabe que:
- Hay valores ausentes en las columnas `days_employed` y `total_income`
- Los valores de la columna `days_employed` son de tipo Float y deberian ser Int
- La columna `dob_years` se refiere a la edad del cliente, el nombre de la columna no es muy claro

Al ver los primeros renglones se puede observar que:
- la columna `days_employed`	tiene valores negativos
- la columna `education` tiene textos iguales en mayusculas y minusculas cosa que los hace diferentes cuando no lo son
- la columna `purpose` tiene propositos iguales escritos de diferentes formas

In [929]:
#Se cambia el nombre de la columna 'dob_years' que se refiere a la edad del cliente para hacerla más clara
df = df.rename(columns={'dob_years':'client_age'})
df.keys()

Index(['children', 'days_employed', 'client_age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [930]:
# Obtener información sobre los datos
df.isna().sum()

children               0
days_employed       2174
client_age             0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Como se observo anteriormente, hay valores ausentes en las columnas `days_employed` y `total_income`.
En ambas columnas faltan el mismo número de datos.

In [931]:
df.isna().sum()/df.count()*100

children             0.000000
days_employed       11.234562
client_age           0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        11.234562
purpose              0.000000
dtype: float64

Más del 11% de los datos de las columnas `days_employed` y `total_income` estan ausentes

In [932]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
df_filtered_isna = df[df['days_employed'].isna()==True]
df_filtered_isna

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Aparentemente todas las filas con valores ausentes de la columna `days_employed` también tienen valores ausentes en la columna `total_income`.



In [933]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
df_filtered_isna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   client_age        2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


**Conclusión intermedia**

El número de filas en la tabla filtrada coincide con el número de valores ausentes, esto confirma que las filas con valores ausentes en la columna `days_employed` también tienen valores ausentes en la columna `total_income`.
Esto podría deberse a que las personas que tienen 0 días trabajando, no tienen ingresos.

Se analiza esta hipótesis y se procede a calcular el porcentaje de los valores ausentes en comparación con el conjunto de datos completo para saber si la porción es considerablemente grande y saber si afecta el análisis.

In [934]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
df_filtered_isna.groupby('income_type')['children'].count()

income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: children, dtype: int64

Se observa que no hay clientes con `income_type` de 'unemployed' por lo que la hipotesis anterior no es correcta.


**Posibles razones por las que hay valores ausentes en los datos**

Es posible que no todos los clientes hayan dado la información de la columna `days_employed` por que no estaban seguros de qué responder. Asimismo, no todas las personas se sienten cómodas informando cuanto ganan al mes, cosa que provocó valores ausentes en la columna `total_income`. Esto también podría deberse a un error a la hora de capturar los datos.

Se sabe que más del 11% de las filas de la tabla no tienen valores en la columna `days_employed` ni en `total_income`, cosa que afecta el analisis, por lo que hay que llenar los valores ausentes con valores representativos para poder continuar con el análisis. 


## Transformación de datos

Se anlizan los valores de cada columna para encontrar posibles problemas en ellas como duplicados y corrección de datos.

Se analiza la columna `education`



In [936]:
#distribución de los valores en la columna educación
df['education'].sort_values().unique()

array(["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE',
       'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education',
       'Some College', "bachelor's degree", 'graduate degree',
       'primary education', 'secondary education', 'some college'],
      dtype=object)

Hay datos duplicados en la columna anterior debido a que unos estan en mayúsculas y otros en minúsculas.


In [937]:
#Establecer datos de la columna en minúsculas
df['education'] = df['education'].str.lower()

In [938]:
# Comprobar cambios
df['education'].sort_values().unique()

array(["bachelor's degree", 'graduate degree', 'primary education',
       'secondary education', 'some college'], dtype=object)

Se analiza la columna `children`

In [939]:
#distribución de los valores en la columna children
df.groupby('children')['education'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: education, dtype: int64

Según los datos, hay clientes con -1 hijos y hay una cantidad significativa de clientes con 20 hijos cosa que es poco probable.
Se determinará el porcentaje de estos datos problemáticos.

In [2]:
f'El porcentaje de datos problematicos es {((47+76)/21525):.2%}'

'El porcentaje de datos problematicos es 0.57%'

El porcentaje de datos es mínimo por lo que se pueden eliminar estas filas sin afectar el análisis.

In [941]:
#se eliminan las filas donde el número de hijos es de -1 ó 20
df =  df [df['children'] != -1]
df =  df [df['children'] != 20]

In [942]:
# Comprobar datos de columna children
df.groupby('children')['education'].count()

children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: education, dtype: int64

Se analiza la columna `days_employed`

Se sabe que la columna es de tipo Float y no Int, por lo que podría haber valores decimales. Asimismo, se observaron valores negativos. 

In [943]:
# Se examinan datos problemáticos en days_employed
df.groupby('days_employed')['education'].count()

days_employed
-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: education, Length: 19240, dtype: int64

In [944]:
#Se buscan numeros negativos
neg_days = df[df['days_employed']<0]['days_employed'].count()
f'el porcentaje de números negativos en la columna days_employed es de {(neg_days/total_rows):.2%}'

'el porcentaje de números negativos en la columna days_employed es de 73.44%'

La cantidad de datos problemáticos es considerablemente alta, esto podría deberse a problemas técnicos. 

No se pueden eliminar estas filas problemáticas así que se asumira que no hay valores negativos cambiando todos los valores a positivos.

In [945]:
#Se abordan los datos problematicos: los valores positivos se hacen negativos y se eliminan los decimales
df['days_employed'] = df['days_employed'].abs()

In [946]:
# Comprobar datos de columna days_employed
df[df['days_employed']<0]['days_employed'].count()

0

Se analiza la columna `client_age`

In [947]:
#Se examina columna client_age
df.groupby('client_age')['education'].count()

client_age
0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    461
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: education, dtype: int64

In [948]:
#Se obtiene el valor de datos problematicos en la edad, que son las filas con clientes de edad 0.
f'el porcentaje de valores de edad de 0 en la columna client_age es de {(101/total_rows):.2%}'

'el porcentaje de valores de edad de 0 en la columna client_age es de 0.47%'

El porcentaje de clientes con edad de 0 no es alto, por lo que estos valores se cambian por un valor representativo

In [949]:
#Corregir datos de columna client_age
years_mean = df['client_age'].mean().round(decimals=2)
df['client_age'] = df['client_age'].replace(0, years_mean)

In [950]:
# Comprobar datos de columna client_age
df.groupby('client_age')['education'].count()

client_age
19.0     14
20.0     51
21.0    110
22.0    183
23.0    252
24.0    263
25.0    356
26.0    406
27.0    490
28.0    501
29.0    543
30.0    536
31.0    556
32.0    506
33.0    577
34.0    597
35.0    614
36.0    553
37.0    531
38.0    595
39.0    572
40.0    603
41.0    603
42.0    592
43.0    510
43.3    100
44.0    543
45.0    494
46.0    469
47.0    480
48.0    536
49.0    505
50.0    509
51.0    446
52.0    483
53.0    457
54.0    476
55.0    441
56.0    482
57.0    457
58.0    461
59.0    441
60.0    376
61.0    353
62.0    351
63.0    268
64.0    263
65.0    194
66.0    183
67.0    167
68.0     99
69.0     83
70.0     65
71.0     58
72.0     33
73.0      8
74.0      6
75.0      1
Name: education, dtype: int64

Se analiza la columna `family_status`

In [951]:
#Se revisa la columna 'family_status' 
df.groupby('family_status')['education'].count()

family_status
civil partnership     4160
divorced              1189
married              12302
unmarried             2799
widow / widower        952
Name: education, dtype: int64

No se encuentran valores extraños en la columna `family_status`

Se analiza la columna `gender`

In [952]:
#Se revisa la columna 'gender' 
df.groupby('gender')['education'].count()

gender
F      14154
M       7247
XNA        1
Name: education, dtype: int64

In [953]:
#Se encuentra un valor extraño, como no se sabe si es hombre o mujer se establece como desconocido.
df['gender'] = df['gender'].replace('XNA', 'unknown')

In [954]:
#Se comprueba el cambio
df.groupby('gender')['education'].count()

gender
F          14154
M           7247
unknown        1
Name: education, dtype: int64

Se analiza la columna `income_type`



In [955]:
#Se examina columna income_type
df.groupby('income_type')['education'].count()

income_type
business                        5054
civil servant                   1453
employee                       11050
entrepreneur                       2
paternity / maternity leave        1
retiree                         3839
student                            1
unemployed                         2
Name: education, dtype: int64

No se encuentran datos extraños en la columna `income_type`

Se analiza la columna `total_income`

In [956]:
#Se examina columna total_income
df.groupby('total_income')['education'].count()

total_income
3306.762      1
3392.845      1
3418.824      1
3471.216      1
3503.298      1
             ..
273809.483    1
274402.943    1
276204.162    1
352136.354    1
362496.645    1
Name: education, Length: 19237, dtype: int64

No se encuentran valores negativos o extraños en esta columna.

Se analiza la columna `purpose`

In [957]:
df['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

En la columna de proposito, hay datos que se refieren a lo mismo pero estan escritos de diferente forma por lo que se modifican

In [958]:
#Función que recibe una lista de valores incorrectos y el valor correcto
def replace_wrong_values(wrong_values, correct_value): 
    for wrong_value in wrong_values:
        df['purpose'] = df['purpose'].replace(wrong_value, correct_value) 

In [959]:
education_list = ['education', 'getting an education', 'getting higher education','going to university','profile education',
                  'supplementary education','to become educated','to get a supplementary education','university education']
car_list = ['buying a second-hand car', 'buying my own car', 'car purchase', 'cars', 'second-hand car purchase', 'to buy a car',
            'to own a car', 'purchase of a car']
house_list = ['purchase of my own house','purchase of the house','purchase of the house for my family']
wedding_list = ['having a wedding', 'to have a wedding','wedding ceremony']


replace_wrong_values(education_list, 'education')
replace_wrong_values(car_list,'car')
replace_wrong_values(house_list,'purchase of house')
replace_wrong_values(wedding_list,'wedding')
df['purpose'] = df['purpose'].replace('transactions with my real estate', 'real estate transactions') 

In [960]:
df['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying property for renting out',
       'car', 'construction of own property', 'education', 'housing',
       'housing renovation', 'housing transactions', 'property',
       'purchase of house', 'real estate transactions',
       'transactions with commercial real estate', 'wedding'],
      dtype=object)

Los datos únicos de propósito se reducen sin mezclar propositos similares

A continuación se analizan los datos en busca de duplicados

In [961]:
# Comprobar los duplicados
df.duplicated().sum()

185

Se encontraron filas con valores aparentemente duplicados; sin embargo, no se tiene una identificación por cliente cosa que no nos permite saber si se duplicaron datos de un solo cliente o si simplemente hay diferentes clientes con los mismos valores.

In [962]:
#informacion del DataFrame modificado
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   client_age        21402 non-null  float64
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      19240 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


El nuevo conjunto de datos es más chico.
Se han eliminado filas con datos extraños como lo son un número de hijos negativo y muy alto(20), se han reducido las categorías de la columna `education` y `purpose` ya que habia datos iguales escritos de diferente forma.

El cambio más grande hecho, fue cambiar el 73.44% de los datos de la columna `days_employed` ya que estos eran negativos y se asume que son positivos.

# Trabajar con valores ausentes

### Restaurar valores ausentes en `total_income`



Hay valores ausentes en las columnas `days_employed` y `total_income`. Al no ser pocas filas, hay que completarlas. Se analizan las características de los ingresos para determinar cual es la mejor referencia para completar las filas faltantes. 


In [964]:
#función que calcula la categoría de edad
def age_category(age):
    if age <20: return 'under 20'
    elif age <30: return '20s'
    elif age <40: return '30s'
    elif age <50: return '40s'
    elif age <60: return '50s'
    elif age <70: return '60s'
    else: return '70 and above'

In [965]:
#Se prueba la función
prueba = [15,20,35,44,56,60,71]
for i in prueba:
    print(age_category(i))

under 20
20s
30s
40s
50s
60s
70 and above


In [966]:
#Se crea una nueva columna basada en la función
df ['age_category'] = df ['client_age'].apply(age_category)

In [967]:
# Comprobar valores de la nueva columna
df.head(10)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of house,20s
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40s
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,50s
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,30s
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of house,40s


Los factores de los que suelen depender los ingresos son la edad, el nivel de educación y el tipo de ingreso



Se crea una tabla sin valores ausentes para analizar los factores mencionados

In [968]:
#tabla sin valores ausentes
df_filtered_full = df[df['days_employed'].isna()==False]
df_filtered_full.head()

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s


In [969]:
#media de ingresos totales por categoría de edad
df_filtered_full.groupby('age_category')['total_income'].mean()

age_category
20s             25585.600814
30s             28314.525654
40s             28514.403672
50s             25807.707523
60s             23236.985508
70 and above    20125.658331
under 20        16993.942462
Name: total_income, dtype: float64

In [970]:
#mediana de ingresos totales por categoría de edad
df_filtered_full.groupby('age_category')['total_income'].median()

age_category
20s             22807.0365
30s             24699.5815
40s             24770.4920
50s             22195.8470
60s             19811.3260
70 and above    18751.3240
under 20        14934.9010
Name: total_income, dtype: float64

In [971]:
#media de ingresos totales por nivel de educación
df_filtered_full.groupby(['education'])['total_income'].mean().sort_values()

education
primary education      21144.882211
secondary education    24587.986103
graduate degree        27960.024667
some college           29039.304112
bachelor's degree      33167.337459
Name: total_income, dtype: float64

In [972]:
#porcentaje de personas por nivel educativo
df_filtered_full.groupby('education')['total_income'].count().sort_values()/total_rows*100

education
graduate degree         0.027875
primary education       1.212544
some college            3.121951
bachelor's degree      21.816492
secondary education    63.205575
Name: total_income, dtype: float64

Al analizar el ingreso por nivel de educación, se observa que mientras mayor el grado de estudios, mayor es el ingreso.
Hay informción de las distintas categorías sin embargo no es mucha.

In [977]:
#Se examinan los ingresos en función del tipo de ingreso
df_filtered_full.groupby('income_type')['total_income'].mean().sort_values()

income_type
paternity / maternity leave     8612.661000
student                        15712.260000
unemployed                     21014.360500
retiree                        21951.754892
employee                       25818.401986
civil servant                  27318.902910
business                       32413.731796
entrepreneur                   79866.103000
Name: total_income, dtype: float64

In [978]:
df_filtered_full.groupby('income_type')['total_income'].count().sort_values()/total_rows*100

income_type
entrepreneur                    0.004646
paternity / maternity leave     0.004646
student                         0.004646
unemployed                      0.009292
civil servant                   6.072009
retiree                        15.930314
business                       21.138211
employee                       46.220674
Name: total_income, dtype: float64

Al agrupar el tipo de ingreso y obtener la media del total de ingresos, se observan diferencias; sin embargo al ver la cantidad de personas que hay en las categorias de `entrepreneur`, `paternity / maternity leave`, `student` y `unemployed`, cada una representa menos del 1% por lo que no es una buena referencia al tener poca información sobre múltiples categorias.

Se establece la mediana de cada categoría de edad como el valor representativo para los datos ausentes, ya que no se tiene mucha información de distintas catgorías del nivel de educación y del tipo de ingreso. Se toma la mediana de cada grupo debido a la presencia de valores atípicos.


In [984]:
#Función para rellenar ingreso por categoria de edad
def fill_total_income2(row):
    if row.isna()['total_income']:
        if row['age_category'] == '20s': row['total_income']   = 22807.0365
        elif row['age_category'] == '30s': row['total_income'] = 24699.5815
        elif row['age_category'] == '40s': row['total_income'] = 24770.4920
        elif row['age_category'] == '50s': row['total_income'] = 22195.8470
        elif row['age_category'] == '60s': row['total_income'] = 19811.3260
        elif row['age_category'] == '70 and above': row['total_income'] = 18751.3240
        elif row['age_category'] == 'under 20': row['total_income'] = 14934.9010
            
    return row

In [985]:
# se prueba la función
#Valores que la función espera
row_values1 = [float('nan'),'60s', 'secondary education']
row_values2 = [100,'20s', 'some college']
row_values3 = [float('nan'),'70 and above', 'secondary education']

#Columnas que la función espera
row_columns = ['total_income','age_category', 'education']
#Crear fila ficticia 
row1 = pd.Series(data=row_values1, index=row_columns)
row2 = pd.Series(data=row_values2, index=row_columns)
row3 = pd.Series(data=row_values3, index=row_columns)

fill_total_income2(row1)

total_income              19811.326
age_category                    60s
education       secondary education
dtype: object

In [986]:
#Aplicar función
df = df.apply(fill_total_income2, axis = 1)

In [987]:
#Vista previa del dataframe
df.head(13)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of house,20s
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40s
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,50s
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,30s
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of house,40s


In [988]:
#información general del dataframe modificado
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   client_age        21402 non-null  float64
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
 12  age_category      21402 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB


La columna `total_income` ya no tiene valores ausentes

###  Restaurar valores en `days_employed`



 Al igual que en la columna anterior, se llenan los valores ausentes en `days_employed` tomando en en cuenta los parámetros de la edad

In [989]:
# Distribución de las medianas de `days_employed` en función del parámetro edad
df_filtered_full.groupby('age_category')['days_employed'].median()

age_category
20s               1007.207074
30s               1601.919871
40s               2109.314691
50s               4823.464469
60s             354932.869424
70 and above    361336.993449
under 20           724.492610
Name: days_employed, dtype: float64

In [990]:
# Distribución de las medias de `days_employed` en función del parámetro edad
df_filtered_full.groupby('age_category')['days_employed'].mean()

age_category
20s               1970.099128
30s               4168.767451
40s              13375.461164
50s             133225.884547
60s             283789.958296
70 and above    320819.151927
under 20           633.678086
Name: days_employed, dtype: float64

Se usan las medianas por la presencia  valores atípicos

In [991]:
#Función para rellenar columna days_employed por categoria de edad
def fill_days_employed(row):
    if row.isna()['days_employed']:
        if row['age_category'] == '20s':             row['days_employed']= 1007.207074
        elif row['age_category'] == '30s':           row['days_employed']= 1601.919871
        elif row['age_category'] == '40s':           row['days_employed']= 2109.314691
        elif row['age_category'] == '50s':           row['days_employed']= 4823.464469
        elif row['age_category'] == '60s':           row['days_employed']= 354932.869424
        elif row['age_category'] == '70 and above':  row['days_employed']= 361336.993449
        elif row['age_category'] == 'under 20':      row['days_employed']= 724.492610
    
    return row

In [992]:
#Se prueba la función

#Valores que la función espera
row_values1 = [float('nan'),'20s', 'business']
row_values2 = [100,'20s', 'business']
row_values3 = [float('nan'),'70 and above', 'retiree']

#Columnas que la función espera
row_columns = ['days_employed','age_category', 'income_type']
#Crear fila ficticia 
row1 = pd.Series(data=row_values1, index=row_columns)
row2 = pd.Series(data=row_values2, index=row_columns)
row3 = pd.Series(data=row_values3, index=row_columns)

fill_days_employed(row3)

days_employed    361336.993449
age_category      70 and above
income_type            retiree
dtype: object

In [993]:
# Aplicar la función a cada fila
df = df.apply(fill_days_employed, axis = 1)

In [994]:
#vista previa del nuevo DF
df.head(13)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of house,20s
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40s
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,50s
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,30s
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of house,40s


In [995]:
#información general del nuevo DF
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   client_age        21402 non-null  float64
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
 12  age_category      21402 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB


Todas las columnas tienen el mismo número de datos, no hay valores ausentes

## Clasificación de datos
Nuevamente, hay que responder las las siguientes preguntas:
- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?

Para ello hay que clasificar ciertos datos como:
- los hijos
- el estado civil
- el nivel de ingresos
- el propósito del prestamo

En cuanto a los datos categóricos, para la última hipótesis, se clasifican los propósitos

In [996]:
#valores de la columna purpose
df['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying property for renting out',
       'car', 'construction of own property', 'education', 'housing',
       'housing renovation', 'housing transactions', 'property',
       'purchase of house', 'real estate transactions',
       'transactions with commercial real estate', 'wedding'],
      dtype=object)

Se separan los propósitos en grupos principales que son:
- compra de un terreno y/o construcción
- compra de un vehiculo
- educación
- boda

In [997]:
#función para clasificar los propositos
def purpose_category(data):
    buy_build = ['buy commercial real estate','buy real estate','buy residential real estate','buying property for renting out',
                 'purchase of house','building a property','building a real estate','construction of own property','housing', 
                 'housing renovation', 'housing transactions', 'property','real estate transactions',
                 'transactions with commercial real estate']
     
    if data in buy_build:     return 'buy or build property'
    elif data == 'car':       return 'car'
    elif data == 'education': return 'education'
    elif data == 'wedding':   return 'wedding'

In [998]:
#Crear columna con las categorías
df ['purpose_category'] = df ['purpose'].apply(purpose_category)
df.head(10)

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s,buy or build property
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s,car
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s,buy or build property
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s,education
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s,wedding
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of house,20s,buy or build property
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40s,buy or build property
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,50s,education
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,30s,wedding
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of house,40s,buy or build property


En cuanto a los datos númericos, una de las clasificaciones es separar los clientes que no tienen hijos de los que si tienen para poder analizar a fondo la primer hipótesis.

In [999]:
#función para clasificacion de hijos
def one_or_more_children(data):
    if data == 0: return 'no'
    else: return 'yes'

In [1000]:
#Aplicar funcion de hijos al DF
df['children>=1'] = df['children'].apply(one_or_more_children)

In [1001]:
#Comprobando columna nueva
df.head()

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category,children>=1
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s,buy or build property,yes
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s,car,yes
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s,buy or build property,no
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s,education,yes
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s,wedding,no


In [1002]:
#Estadísticas resumidas de la columna
df.groupby('children')['children'].count()

children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Otra clasificación que hay que tomar en cuenta para comprobar la tercer hipótesis es el nivel de ingresos al separar en categorías el ingreso total

In [1003]:
#Función para clasificar el ingreso total en rangos
def income_level(data):
    if data < 10000:  return 'less than 10,000'
    elif data <20000: return '10,000 - 19,999'
    elif data <30000: return '20,000 - 29,999'
    else: return 'more than 30,000'

In [1004]:
#Crear columna con categorías de ingresos totales
df['income_level'] = df['total_income'].apply(income_level)
df.head()

,children,days_employed,client_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category,children>=1,income_level
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of house,40s,buy or build property,yes,"more than 30,000"
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,30s,car,yes,"10,000 - 19,999"
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of house,30s,buy or build property,no,"20,000 - 29,999"
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,30s,education,yes,"more than 30,000"
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50s,wedding,no,"20,000 - 29,999"


In [1005]:
# Contar los valores de cada categoría para ver la distribución
df.groupby('income_level')['children'].count().sort_values()

income_level
less than 10,000     921
more than 30,000    5888
10,000 - 19,999     6674
20,000 - 29,999     7919
Name: children, dtype: int64

## Comprobación de las hipótesis



**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [1006]:
#Se filtran los clientes que si han incumplido
df_debt1 = df[df['debt']==1]
df_children_debt1 = df_debt1.groupby('children')['debt'].count()

# Calcular la tasa de incumplimiento en función del número de hijos
total_per_children = df.groupby('children')['days_employed'].count()
(df_children_debt1 / total_per_children) *100

children
0    7.512898
1    9.215442
2    9.440389
3    8.181818
4    9.756098
5         NaN
dtype: float64

Los datos anteriores muestran el porcentaje de clientes que han incumplido prestamos por cantidad de hijos; es decir el 7.5% de los clientes con 0 hijos han incumplido el pago de prestamos y así sucesivamente.

In [1007]:
#Calcular la tasa de incumplimiento en función de si tiene o no hijos
df_children_debt2 = df_children_debt.groupby('children>=1')['debt'].count()
df_children_debt2/21402 *100

children>=1
no     4.966826
yes    3.125876
Name: debt, dtype: float64

**Conclusión**

Al analizar la tasa de incumplimiento separando a los clientes por la cantidad de hijos que tienen, todos los porcentajes oscilan entre el 7 y el 10%, de su correspondiente categoría, por lo que aparentemente no influye. 

Al analizar la tasa tomando en cuenta si los clientes tienen o no hijos, se observa que los que no tienen hijos tienden casi un 2% más a incumplir en el pago que los que si tienen hijos; esto es que tienen incumplimiento de pago:
- el 4.97% de los clientes sin hijos
- el 3.12% de los clientes con hijos

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**



In [1008]:
#Analizar los datos del estado familiar y los pagos a tiempo
df_family_st_debt1 = df_debt1.groupby('family_status')['debt'].count()

# Calcular la tasa de incumplimiento basada en el estado familiar
total_per_family_st = df.groupby('family_status')['debt'].count()
(df_family_st_debt1 / total_per_family_st *100).sort_values()

family_status
widow / widower      6.617647
divorced             7.064760
married              7.535360
civil partnership    9.254808
unmarried            9.753483
Name: debt, dtype: float64

**Conclusión**

Al analizar el estatus familiar se observa que los viudos tienen a pagar aproximadamente un 3% más que los que se encuentran en una union civil y que los que no estan casados.

Tomando en cuenta el análisis anterior, se pueden separar en 3 categorías, donde el incumplimiento de pago lo tienen: 
- el 6.62% de los viudos y viudas
- el 7-7.54% de los divorciados y casados
- el 9-9.75% de los de union civil y solteros

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [1009]:
# Analizar los datos del nivel de ingresos y los pagos a tiempo
df_income_level_debt1 = df_debt1.groupby('income_level')['debt'].count()

# Calcular la tasa de incumplimiento basada en el nivel de ingresos
total_per_income_level = df.groupby('income_level')['debt'].count()
(df_income_level_debt1/total_per_income_level*100).sort_values()

income_level
less than 10,000    6.297503
more than 30,000    7.370924
10,000 - 19,999     8.435721
20,000 - 29,999     8.549059
Name: debt, dtype: float64

**Conclusión**

En base a este análisis se afirma que tienen incumplimiento de pago:
- el 6.29% de los clientes que ganan menos de 10,000 
- el 7.37% de los que ganan más de 30,000
- el 8.44% - 8.55% de los cliente que ganan entre 10,000 y 29,999

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [1010]:
# Analizar los datos de cada propósito del crédito y los pagos a tiempo
df_purpose_debt1 = df_debt1.groupby('purpose_category')['debt'].count()

# Calcular la tasa de incumplimiento basada en el propósito
total_per_purpose = df.groupby('purpose_category')['debt'].count()
(df_purpose_debt1/total_per_purpose*100).sort_values()

purpose_category
buy or build property    7.235622
wedding                  7.830552
education                9.231924
car                      9.328358
Name: debt, dtype: float64

**Conclusión**

Tomando en cuenta lo anterior, se observa que en cuanto al propósito tienen incumplimiento de pago:
- el 7.24% - 7.83 de los clientes cuyo propósito está relacionado con terrenos y bodas
- el 9.23% - 9.33% de los clientes cuyo propósito está relacionado con educación y vehiculos

# Conclusión general 

En cuanto a los datos usados para el análisis, se puede afirmar que:

- menos del 1% de los valores en la columna `dob_years` (cambiada a `client_age`) y `children` eran erróneos
- hay un error en captura de la columna `days employed` pues más del 70% de estos datos eran negativos y se asumieron positivos
- hay un error de captura en las columnas `days employed` y `total_income` pues faltaba el 11% de sus datos en las mismas filas
- En las columnas categóricas como `education` y `purpose` sería útil el darle opciones a quien llene los datos para evitar tener respuestas iguales escritas de diferente forma pues fue necesario hacerles modificaciones y clasificarlas 

En cuanto a las preguntas planteadas, se determinó lo siguiente:

**Clientes con o sin hijos**

- El 4.97% de los clientes sin hijos tienden a incumplir a diferencia del 3.12% de los clientes con hijos; es decir casi un 2% de diferencia.

**Estatus familiar de los clientes**

- el 6.62% de los viudos y viudas tienden a incumplir
- entre el 7% y 7.54% de los divorciados y casados tienden a incumplir
- entre el 9% y 9.75% de los de union civil y solteros tienden a incumplir. Esto es una diferencia de casi 3% entre los primeros y los últimos mencionados.

**Nivel de ingreso de los clientes**

- el 6.29% de los clientes que ganan menos de 10,000 tienden a incumplir
- el 7.37% de los que ganan más de 30,000 tienden a incumplir
- entre el 8.44% y 8.55% de los cliente que ganan entre 10,000 y 29,999 tienden a incumplir. Hay una diferencia de más de 2% entre la primer y última categoría.

**Propósito del prestamo de los clientes**

- aproximadamente 7% de los clientes cuyo propósito está relacionado con terrenos y bodas tienden a incumplir
- aproximadamente 9% de los clientes cuyo propósito está relacionado con educación y vehiculos tienden a incumplir. La tasa de diferencia en base a los propositos es de aproximadamente 2%.

